In [ ]:
%cd /content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit

/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit


In [ ]:
!pip install mxnet

In [ ]:
import os, sys, random
import xml.etree.ElementTree as ET
import mxnet as mx
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile,copy
import sys
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
testing_annotation_loc = '/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/csv_folder/test-annotations-bbox.csv'
testing_annotation_file = pd.read_csv(testing_annotation_loc)
print("records present for training : ",testing_annotation_file.shape)
testing_annotation_file.head()

records present for training :  (625282, 13)


,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,freeform,/m/07j7r,1,0.071905,0.145346,0.206591,0.391306,0,1,1,0,0
1,000026e7ee790996,freeform,/m/07j7r,1,0.439756,0.572466,0.264153,0.435122,0,1,1,0,0
2,000026e7ee790996,freeform,/m/07j7r,1,0.668455,1.000000,0.000000,0.552825,0,1,1,0,0
3,000062a39995e348,freeform,/m/015p6,1,0.205719,0.849912,0.154144,1.000000,0,0,0,0,0
4,000062a39995e348,freeform,/m/05s2s,1,0.137133,0.377634,0.000000,0.884185,1,1,0,0,0


In [ ]:
class_descriptions_file='/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv'
class_descriptions = pd.read_csv(class_descriptions_file, header=None)
print(class_descriptions.shape)
class_descriptions.head()

(601, 2)


,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [ ]:
car_pd = class_descriptions[class_descriptions[1]=='Car']
bus_pd = class_descriptions[class_descriptions[1]=='Bus']
train_pd = class_descriptions[class_descriptions[1]=='Train']
train_pd

,0,1
392,/m/07jdr,Train


In [ ]:
label_name_car = car_pd[0].values[0]
label_name_bus = bus_pd[0].values[0]
label_name_train = train_pd[0].values[0]
label_name_train

'/m/07jdr'

In [ ]:
car_bbox = testing_annotation_file[testing_annotation_file['LabelName']==label_name_car]
bus_bbox = testing_annotation_file[testing_annotation_file['LabelName']==label_name_bus]
train_bbox = testing_annotation_file[testing_annotation_file['LabelName']==label_name_train]
train_bbox.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
3527,0118804d8fedc568,freeform,/m/07jdr,1,0.014564,0.967998,0.137014,0.859604,0,1,0,0,0
5659,01e94a3389de3378,freeform,/m/07jdr,1,0.011531,0.999452,0.099968,0.959652,0,0,0,0,0
7352,0284916969f261ee,freeform,/m/07jdr,1,0.000390,1.000000,0.266551,0.691335,0,1,0,0,0
9240,032438d1cb5c491e,freeform,/m/07jdr,1,0.010806,0.914355,0.100563,0.820781,0,0,0,0,0
14087,04fc06e1cd489eff,freeform,/m/07jdr,1,0.154128,0.717966,0.181632,0.582038,0,0,0,0,0


In [ ]:
print('There are %d car in the dataset' %(len(car_bbox)))
print('There are %d bus in the dataset' %(len(bus_bbox)))
print('There are %d train in the dataset' %(len(train_bbox)))

There are 28737 car in the dataset
There are 353 bus in the dataset
There are 315 train in the dataset


In [ ]:
car_img_id = car_bbox['ImageID']
bus_img_id = bus_bbox['ImageID']
train_img_id = train_bbox['ImageID']
train_img_id.count()
train_img_id

3527      0118804d8fedc568
5659      01e94a3389de3378
7352      0284916969f261ee
9240      032438d1cb5c491e
14087     04fc06e1cd489eff
                ...       
621068    fe28099adc793b28
621069    fe28099adc793b28
623133    ff025b2ab3ed9942
623980    ff69e5ebb58b9435
624617    ffb023607751cf52
Name: ImageID, Length: 315, dtype: object

In [ ]:
concated_df=pd.concat([car_bbox, bus_bbox, train_bbox], axis=0)
concated_df

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
6,0000c64e1253d68f,freeform,/m/0k4j,1,0.000000,0.513534,0.321356,0.689661,0,1,0,0,0
7,0000c64e1253d68f,freeform,/m/0k4j,1,0.016515,0.268228,0.299368,0.462906,1,0,0,0,0
8,0000c64e1253d68f,freeform,/m/0k4j,1,0.481498,0.904376,0.232029,0.489017,1,0,0,0,0
9,0000c64e1253d68f,freeform,/m/0k4j,1,0.751517,1.000000,0.332350,0.658053,0,1,0,0,0
282,001547e7ef44a7d8,freeform,/m/0k4j,1,0.000000,0.051603,0.262482,0.304314,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
621068,fe28099adc793b28,freeform,/m/07jdr,1,0.000000,0.248337,0.088865,0.666749,1,1,0,0,0
621069,fe28099adc793b28,freeform,/m/07jdr,1,0.126542,0.889874,0.105136,0.867960,0,0,0,0,0
623133,ff025b2ab3ed9942,freeform,/m/07jdr,1,0.000000,0.929646,0.548821,1.000000,0,1,0,0,0
623980,ff69e5ebb58b9435,freeform,/m/07jdr,1,0.000000,0.999920,0.371874,0.737122,1,1,0,0,0


In [ ]:
concated_df['ClassName']=concated_df['LabelName']
concated_df['ClassName']

6          /m/0k4j
7          /m/0k4j
8          /m/0k4j
9          /m/0k4j
282        /m/0k4j
            ...   
621068    /m/07jdr
621069    /m/07jdr
623133    /m/07jdr
623980    /m/07jdr
624617    /m/07jdr
Name: ClassName, Length: 29405, dtype: object

In [ ]:
mapping = {
    label_name_car: 'Car',
    label_name_bus: 'Bus',
    label_name_train: 'Train',
}

concated_df['ClassName'] = [mapping[word] for word in concated_df['ClassName']]
concated_df['ClassName']

concated_df

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,ClassName
6,0000c64e1253d68f,freeform,/m/0k4j,1,0.000000,0.513534,0.321356,0.689661,0,1,0,0,0,Car
7,0000c64e1253d68f,freeform,/m/0k4j,1,0.016515,0.268228,0.299368,0.462906,1,0,0,0,0,Car
8,0000c64e1253d68f,freeform,/m/0k4j,1,0.481498,0.904376,0.232029,0.489017,1,0,0,0,0,Car
9,0000c64e1253d68f,freeform,/m/0k4j,1,0.751517,1.000000,0.332350,0.658053,0,1,0,0,0,Car
282,001547e7ef44a7d8,freeform,/m/0k4j,1,0.000000,0.051603,0.262482,0.304314,1,1,0,0,0,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621068,fe28099adc793b28,freeform,/m/07jdr,1,0.000000,0.248337,0.088865,0.666749,1,1,0,0,0,Train
621069,fe28099adc793b28,freeform,/m/07jdr,1,0.126542,0.889874,0.105136,0.867960,0,0,0,0,0,Train
623133,ff025b2ab3ed9942,freeform,/m/07jdr,1,0.000000,0.929646,0.548821,1.000000,0,1,0,0,0,Train
623980,ff69e5ebb58b9435,freeform,/m/07jdr,1,0.000000,0.999920,0.371874,0.737122,1,1,0,0,0,Train


In [ ]:
test_df= concated_df.loc[:,['ImageID','XMin','XMax','YMin','YMax','ClassName']]
test_df

,ImageID,XMin,XMax,YMin,YMax,ClassName
6,0000c64e1253d68f,0.000000,0.513534,0.321356,0.689661,Car
7,0000c64e1253d68f,0.016515,0.268228,0.299368,0.462906,Car
8,0000c64e1253d68f,0.481498,0.904376,0.232029,0.489017,Car
9,0000c64e1253d68f,0.751517,1.000000,0.332350,0.658053,Car
282,001547e7ef44a7d8,0.000000,0.051603,0.262482,0.304314,Car
...,...,...,...,...,...,...
621068,fe28099adc793b28,0.000000,0.248337,0.088865,0.666749,Train
621069,fe28099adc793b28,0.126542,0.889874,0.105136,0.867960,Train
623133,ff025b2ab3ed9942,0.000000,0.929646,0.548821,1.000000,Train
623980,ff69e5ebb58b9435,0.000000,0.999920,0.371874,0.737122,Train


In [ ]:
test_df['ImageID']=test_df['ImageID']+'.jpg'
test_df

,ImageID,XMin,XMax,YMin,YMax,ClassName
6,0000c64e1253d68f.jpg,0.000000,0.513534,0.321356,0.689661,Car
7,0000c64e1253d68f.jpg,0.016515,0.268228,0.299368,0.462906,Car
8,0000c64e1253d68f.jpg,0.481498,0.904376,0.232029,0.489017,Car
9,0000c64e1253d68f.jpg,0.751517,1.000000,0.332350,0.658053,Car
282,001547e7ef44a7d8.jpg,0.000000,0.051603,0.262482,0.304314,Car
...,...,...,...,...,...,...
621068,fe28099adc793b28.jpg,0.000000,0.248337,0.088865,0.666749,Train
621069,fe28099adc793b28.jpg,0.126542,0.889874,0.105136,0.867960,Train
623133,ff025b2ab3ed9942.jpg,0.000000,0.929646,0.548821,1.000000,Train
623980,ff69e5ebb58b9435.jpg,0.000000,0.999920,0.371874,0.737122,Train


In [ ]:
test_df.shape
test_df.to_csv('test.csv')

In [ ]:
test_df['ClassName'].value_counts()

Car      28737
Bus        353
Train      315
Name: ClassName, dtype: int64

In [ ]:
%cd /content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test
!mkdir test
print('Test folder created!!')

/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test
mkdir: cannot create directory ‘test’: File exists
Test folder created!!


In [ ]:
import shutil
folders = ['Car','Bus','Train']
folder = '/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/test'
for i in folders:
  path = '/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/' + i  
  for j in os.listdir(path):
    img = path + '/' + j
    try:
      shutil.copy(img,folder)
      print(img + ' copied!!')
    except IsADirectoryError:
      print(img + 'Is a directory!!')
      continue
    except shutil.SameFileError:      
      continue

/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/005db64b7bfc26bb.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/00a5679940461f10.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/01b52943f00323a0.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/04a8edff0b563d43.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/05b8cb33917d9b22.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/08e9d0b625bf9bd3.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/0a74367f0c70fb25.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/13998fa14da472bf.jpg copied!!
/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/Car/1454

In [ ]:
import os
files = []
path = '/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/test'

for i in os.listdir(path):
  files.append(i)
print("Images found: ",len(files))

Images found:  448


In [ ]:
%cd /content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit
train_df = pd.read_csv('/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/test.csv')
p = '/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/OID/Dataset/test/test/'

found=0
notFound=0
with open("annotation_test.txt", "w+") as f:
  for idx, row in train_df.iterrows():
    n = row['ImageID']
    if n in files:
      found+=1
      img = cv2.imread(p + n)
      #print(img)  #none 
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      x2 = int(row['XMax'] * width)
      y1 = int(row['YMin'] * height)
      y2 = int(row['YMax'] * height)
      
      train_file_path = p
      fileName = os.path.join(train_file_path, row['ImageID'])
      className = row['ClassName']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
    else:
      notFound+=1
print('Found: ',found)
print('Not Found: ',notFound)

/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit
Found:  801
Not Found:  28604


In [1]:
f = open('/content/drive/MyDrive/ColabProjects/Faster-RCNN/OIDv4_ToolKit/annotation_test.txt','r')
lines = f.readlines()

In [ ]:
import cv2
import matplotlib.pyplot as plt

line = lines[0]
temp = line.split(',')

img = cv2.imread(temp[0])
x1 = int (temp[1])
y1 = int (temp[2])
x2 = int (temp[3])
y2 = int (temp[4])
c = temp[5]
img = cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,255),5)
print(img.shape[:2])
print(c)
plt.imshow(img)
plt.show()